In [ ]:
from hypergraph import SyncRunner, Graph, node


@node(output_name="doubled")
def double(x: int) -> int:
    return x * 2


@node(output_name="tripled")
def triple(doubled: int) -> int:
    return doubled * 3


runner = SyncRunner()
graph = Graph([double, triple], name="pipeline")

# RunLog Discovery Flow

Three ways to run the same graph. Each gives you the **same RunLog API** — just accessed differently.

## Step 1: Single run — the baseline

Run the graph once. `result.log` is your RunLog.

In [ ]:
result = runner.run(graph, {"x": 5})
result.log.summary()

In [ ]:
print(result.log)

In [ ]:
result.log.timing

## Step 2: Mapped run — N independent runs

`runner.map()` returns a `MapResult`. Each item has its own RunLog.

In [ ]:
results = runner.map(graph, {"x": [1, 2, 3, 4, 5]}, map_over="x")
results.summary()

`results.log` gives you a `MapLog` — batch overview. Drill into one item with `[i]`:

In [ ]:
print(results.log)

In [ ]:
results.log[0].timing

## Step 3: Nested run — map_over inside a graph

`map_over` wraps the graph as a node and maps it over a list input. 
One `run()` call, one `RunResult` — but how do you see inside?

In [ ]:
inner = Graph([double, triple], name="pipeline")
outer = Graph([inner.as_node().map_over("x")])
result = runner.run(outer, {"x": [1, 2, 3, 4, 5]})

Summary says 1 node — looks flat:

In [ ]:
result.log.summary()

But `print()` reveals `(5 inner)` — there's more inside:

In [ ]:
print(result.log)

`step.log` gives you a `MapLog` (5 items). Drill into one — **same API as Step 1**:

In [ ]:
step = result.log.steps[0]
print(step.log[0])

In [ ]:
step.log[0].timing

In [ ]:
step.log